In [ ]:
#
# First Check, if we are running in Google CoLab
#
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    print('Running on Google CoLab!')
else:
    print('Not running on Google CoLab!')

import os
if IN_COLAB:
    !pip3 install keras-tcn
    if not os.path.exists('/content/bioma-tcn-ae/'):
        print("Repo not cloned yet. Do it now!")
        !git clone https://github.com/MarkusThill/bioma-tcn-ae /content/bioma-tcn-ae/
    else:
        print("Repository already cloned!")

#
# In Google CoLab: Change the working directory to bioma-tcn-ae/src
#
if IN_COLAB and os.getcwd() != "/content/bioma-tcn-ae/src":
  # Print the current working directory
  print("Old working directory: {0}".format(os.getcwd()))

  # Change the current working directory
  os.chdir('/content/bioma-tcn-ae/src')

  # Print the current working directory
  print("New working directory: {0}".format(os.getcwd()))

if IN_COLAB:
    %tensorflow_version 2.x
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
    print('Found GPU at: {}'.format(device_name))

In [ ]:
import numpy
import time
from utilities import select_gpus, plot_results_swat# utilities.py: Contains a few miscellaneous functions 
from tcnae import TCNAE # tcnae.py: Specification of the TCN-AE model
import data_swat 


In [ ]:
train_ts_id = 1 # [1-10]. Train the model on Mackey-Glass time series 1
data_gen = data_swat.DataSwat(window_length = 1050, ratio = 1)
train_data = data_gen.build_data() # Returns a dictionary
train_X = train_data["train_X"] # We only need train_X (input = output) for the training process
print("train_X.shape:", train_X.shape) # A lot of training sequences of length 1050 and dimension 1


In [ ]:
tcn_ae = TCNAE(ts_dimension=train_X.shape[2], dilations=(1, 2, 4, 8, 16)) # Use the parameters specified in the paper
tcn_ae.fit(train_X, train_X, batch_size=8, epochs=10, verbose=1)


In [ ]:
start_time = time.time()

#
# Take the whole time series... Like the training data, the test data is standardized (zero mean and unit variance)
#
test_X = train_data['test_X'] 
test_labels = train_data['test_labels']# We need an extra dimension for the batch-dimension
print("test_X.shape", test_X.shape) # This is one long time series
anomaly_score = tcn_ae.predict(test_X)
print("> Time:", round(time.time() - start_time), "seconds.")


In [ ]:
plot_results_swat(test_X, test_labels, anomaly_score, pl_range = None, plot_signal = False, plot_anomaly_score = True)
